In [ ]:
import pandas as pd
import numpy as np

# Set the file path (Update this if needed)
file_path = "mario_kart_stats - Backup3.xlsx"
sheet_name = "Builds Normalized"

# Load the dataset with correct headers
df = pd.read_excel(file_path, sheet_name=sheet_name, header=1)

# Select key stats for Pareto optimization
key_stats = [
    "BrakeRt_SAND_H Normal",
    "BrakeRt_ICE_H Normal",
    "SlipRt_SAND_H Normal",
    "SlipRt_ICE_H Normal",
    "SMT Length Normal",
    "Drive_SpdMaxFwd_H Normal",
    "Float_SpdMaxFwd_H Normal",
    "SpdMaxFwd_H Normal",
    "Drift_DirRotH Normal",
    "Float_Drift_DirRotH Normal",
    "Drift_DirRotH2 Normal",
    "SpdMaxFwd_H Normal",
    "Invincibility Time Multiplier Normal",
]

# Convert DataFrame to NumPy for fast processing
data = df[key_stats].to_numpy()


def is_pareto_efficient(data):
    """
    Finds Pareto-efficient points (non-dominated rows).
    """
    num_points = data.shape[0]
    pareto_mask = np.ones(num_points, dtype=bool)

    for i in range(num_points):
        if pareto_mask[i]:  # Only check points not already dominated
            pareto_mask &= ~np.all(data >= data[i], axis=1) | np.any(
                data > data[i], axis=1
            )
            pareto_mask[i] = True  # Keep itself

    return pareto_mask


# Get Pareto front
pareto_mask = is_pareto_efficient(data)
pareto_front = df[pareto_mask]

# Function to evaluate trade-offs
def efficiency_score(setups, key_stats):
    """
    Ranks setups based on how much they gain per unit of loss in other stats.
    """
    scores = []
    for i, setup in setups.iterrows():
        total_gain = 0
        total_loss = 0

        for j, other_setup in setups.iterrows():
            if i == j:
                continue  # Don't compare a setup with itself

            # Compute stat-by-stat differences
            diffs = setup[key_stats] - other_setup[key_stats]

            # Gains are positive values, losses are negative values
            gain = diffs[diffs > 0].sum()
            loss = -diffs[diffs < 0].sum()

            # Compute efficiency ratio
            if loss > 0:  # Avoid division by zero
                efficiency = gain / loss
            else:
                efficiency = gain  # If no loss, take pure gain

            total_gain += gain
            total_loss += loss

        # Store the efficiency score
        scores.append((i, total_gain, total_loss, total_gain / (total_loss + 1e-9)))  # Avoid zero division

    # Sort by best efficiency score
    scores.sort(key=lambda x: x[3], reverse=True)
    
    # Extract best setups
    best_indexes = [x[0] for x in scores[:10]]  # Top 10 efficient setups
    return setups.loc[best_indexes]


# Rank setups by trade-off efficiency
best_tradeoff_setups = efficiency_score(pareto_front, key_stats)

# Display results
print("\n=== Best Trade-Off Kart Setups ===")
print(best_tradeoff_setups[["Driver", "Body", "Tire", "Glider"] + key_stats])

# Save results to a new Excel file
output_file = "best_tradeoff_kart_builds.xlsx"
best_tradeoff_setups.to_excel(output_file, index=False)
print(f"\nResults saved to {output_file}")
